# Registry Client Example

This notebook demonstrates how to use the `LocalRegistryClient` to access model information from LangGate's model registry.

In [1]:
import os
from pprint import pprint as pp

# Optional environment variables
# os.environ["LANGGATE_CONFIG"] = "path/to/langgate_config.yaml"
# os.environ["LANGGATE_MODELS"] = "path/to/langgate_models.json"
# os.environ["LOG_LEVEL"] = "debug"

## Initialize the Registry Client

In [2]:
from langgate.registry import LocalRegistryClient

# Initialize the registry client
# LocalRegistryClient is a singleton
client = LocalRegistryClient()

2025-07-09 23:31:54 [debug    ] creating_model_registry_singleton
2025-07-09 23:31:54 [info     ] loaded_config                  config_path=/Users/someuser/langgate/examples/langgate_config.yaml
2025-07-09 23:31:54 [info     ] loaded_model_data              merge_mode=merge model_count=97 models_data_path=/Users/someuser/langgate/packages/registry/src/langgate/registry/data/default_models.json
2025-07-09 23:31:54 [debug    ] initialized_model_registry_singleton config_path=/Users/someuser/langgate/examples/langgate_config.yaml env_file_exists=False env_file_path=/Users/someuser/langgate/examples/.env models_data_path=/Users/someuser/langgate/packages/registry/src/langgate/registry/data/default_models.json
2025-07-09 23:31:54 [debug    ] initialized_base_local_registry_client
2025-07-09 23:31:54 [debug    ] initialized_local_registry_client_singleton


## List Available Models

In [3]:
# List available LLMs
llms = await client.list_llms()
print(f"Available LLMs: {len(llms)}")
for model in llms[:5]:
    print(f"- {model.id}: {model.name}")

print("\n" + "=" * 50 + "\n")

# List available image models
image_models = await client.list_image_models()
print(f"Available Image Models: {len(image_models)}")
for model in image_models[:5]:
    print(f"- {model.id}: {model.name}")

2025-07-09 23:31:57 [debug    ] refreshing_model_caches       
2025-07-09 23:31:57 [debug    ] refreshed_model_caches         image_count=4 llm_count=5
Available LLMs: 5
- openai/gpt-4.1: GPT-4.1
- openai/o3: o3
- openai/o3-high: o3-high
- anthropic/claude-sonnet-4: Claude-4 Sonnet
- anthropic/claude-sonnet-4-reasoning: Claude-4 Sonnet R


Available Image Models: 4
- openai/gpt-image-1: GPT Image 1
- openai/dall-e-3: DALL-E 3
- black-forest-labs/flux-dev: FLUX.1 [dev]
- stability-ai/sd-3.5-large: SD 3.5 Large


## Get Detailed Model Information

In [4]:
# Get a specific LLM
if llms:
    llm_id = llms[0].id
    llm_info = await client.get_llm_info(llm_id)

    print(f"\nLLM Model: {llm_info.name}")
    print(f"Provider: {llm_info.provider.name}")

    print("\nLLM metadata:")
    pp(llm_info.model_dump(exclude_none=True))


LLM Model: GPT-4.1
Provider: OpenAI

LLM metadata:
{'capabilities': {'supports_parallel_tool_calls': True,
                  'supports_prompt_caching': True,
                  'supports_response_schema': True,
                  'supports_system_messages': True,
                  'supports_tool_choice': True,
                  'supports_tools': True,
                  'supports_vision': True},
 'context_window': {'max_input_tokens': 1047576, 'max_output_tokens': 32768},
 'costs': {'cache_read_input_token_cost': Decimal('5E-7'),
           'input_cost_per_token': Decimal('0.000002'),
           'output_cost_per_token': Decimal('0.000008')},
 'description': "GPT-4.1 is the latest iteration of OpenAI's flagship model "
                'with improved capabilities across all domains.',
 'id': 'openai/gpt-4.1',
 'name': 'GPT-4.1',
 'provider': {'id': 'openai', 'name': 'OpenAI'},
 'provider_id': 'openai',
 'updated_dt': datetime.datetime(2025, 7, 9, 22, 31, 54, 428306, tzinfo=datetime.timezon

In [5]:
# Get a specific image model
if image_models:
    image_id = image_models[0].id
    image_info = await client.get_image_model_info(image_id)

    print(f"Image Model: {image_info.name}")
    print(f"Provider: {image_info.provider.name}")

    print("\nImage model metadata:")
    pp(image_info.model_dump(exclude_none=True))

Image Model: GPT Image 1
Provider: OpenAI

Image model metadata:
{'costs': {'image_generation': {'quality_tiers': {'high': {'1024x1024': Decimal('0.167'),
                                                           '1024x1536': Decimal('0.25'),
                                                           '1536x1024': Decimal('0.25')},
                                                  'low': {'1024x1024': Decimal('0.011'),
                                                          '1024x1536': Decimal('0.016'),
                                                          '1536x1024': Decimal('0.016')},
                                                  'medium': {'1024x1024': Decimal('0.042'),
                                                             '1024x1536': Decimal('0.063'),
                                                             '1536x1024': Decimal('0.063')}}},
           'token_costs': {'input_cached_cost_per_token': Decimal('0.0000025'),
                           'input_cost_

## Filtering Models

In [6]:
# Filter models by provider
openai_models = [model for model in llms if model.provider.id == "openai"]
print(f"OpenAI models: {len(openai_models)}")
for model in openai_models[:3]:
    print(f"- {model.id}: {model.name}")

OpenAI models: 3
- openai/gpt-4.1: GPT-4.1
- openai/o3: o3
- openai/o3-high: o3-high


In [7]:
# Filter models by capability
vision_models = [model for model in llms if model.capabilities.supports_vision]
print(f"\nModels with vision capabilities: {len(vision_models)}")
for model in vision_models[:5]:
    print(f"- {model.id}: {model.name}")


Models with vision capabilities: 5
- openai/gpt-4.1: GPT-4.1
- openai/o3: o3
- openai/o3-high: o3-high
- anthropic/claude-sonnet-4: Claude-4 Sonnet
- anthropic/claude-sonnet-4-reasoning: Claude-4 Sonnet R


## Working with Model Costs

In [8]:
# Get cost information for an LLM
if llms:
    llm_id = llms[0].id
    llm_info = await client.get_llm_info(llm_id)

    print(f"Cost information for {llm_info.name}:")
    pp(llm_info.costs.model_dump(exclude_none=True))

Cost information for GPT-4.1:
{'cache_read_input_token_cost': Decimal('5E-7'),
 'input_cost_per_token': Decimal('0.000002'),
 'output_cost_per_token': Decimal('0.000008')}


In [9]:
# Get cost information for an image model
if image_models:
    image_id = image_models[0].id
    image_info = await client.get_image_model_info(image_id)

    print(f"Cost information for {image_info.name}:")
    pp(image_info.costs.model_dump(exclude_none=True))

Cost information for GPT Image 1:
{'image_generation': {'quality_tiers': {'high': {'1024x1024': Decimal('0.167'),
                                                 '1024x1536': Decimal('0.25'),
                                                 '1536x1024': Decimal('0.25')},
                                        'low': {'1024x1024': Decimal('0.011'),
                                                '1024x1536': Decimal('0.016'),
                                                '1536x1024': Decimal('0.016')},
                                        'medium': {'1024x1024': Decimal('0.042'),
                                                   '1024x1536': Decimal('0.063'),
                                                   '1536x1024': Decimal('0.063')}}},
 'token_costs': {'input_cached_cost_per_token': Decimal('0.0000025'),
                 'input_cost_per_token': Decimal('0.00001'),
                 'output_cost_per_token': Decimal('0.00004')}}


In [9]:
# Compare different image model cost structures
print("Image Model Cost Structures:\n")

# Show different pricing models
for model in image_models[:5]:
    model_info = await client.get_image_model_info(model.id)
    print(f"{model_info.name}:")

    if model_info.costs.image_generation.flat_rate:
        print(f"  - Flat rate: ${model_info.costs.image_generation.flat_rate}")
    elif model_info.costs.image_generation.quality_tiers:
        print("  - Quality-based pricing:")
        for quality, sizes in model_info.costs.image_generation.quality_tiers.items():
            print(f"    {quality}:")
            for size, price in sizes.items():
                print(f"      {size}: ${price}")

    # Check if model also has token costs (hybrid pricing)
    if model_info.costs.token_costs:
        print("  - Also has token-based costs:")
        print(f"    Input: ${model_info.costs.token_costs.input_cost_per_token}/token")
        print(
            f"    Output: ${model_info.costs.token_costs.output_cost_per_token}/token"
        )

    print()

Image Model Cost Structures:

GPT Image 1:
  - Quality-based pricing:
    low:
      1024x1024: $0.011
      1024x1536: $0.016
      1536x1024: $0.016
    medium:
      1024x1024: $0.042
      1024x1536: $0.063
      1536x1024: $0.063
    high:
      1024x1024: $0.167
      1024x1536: $0.25
      1536x1024: $0.25
  - Also has token-based costs:
    Input: $0.00001/token
    Output: $0.00004/token

DALL-E 3:
  - Quality-based pricing:
    standard:
      1024x1024: $0.04
      1024x1792: $0.08
      1792x1024: $0.08
    hd:
      1024x1024: $0.08
      1024x1792: $0.12
      1792x1024: $0.12

FLUX.1 [dev]:
  - Flat rate: $0.025

SD 3.5 Large:
  - Flat rate: $0.065

